In [4]:
#default_exp utils.parseyaml

In [5]:
#hide
from nbdev.showdoc import *

In [6]:
#export

from ruamel.yaml import YAML

import logging
import os

logging.basicConfig(level=logging.INFO)
logging.getLogger("azure.core").setLevel(logging.WARNING)
logging.getLogger("urllib3.connectionpool").setLevel(logging.CRITICAL)
logging.getLogger("snowflake.connector").setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

# ``ParseYaml`` & ``yml_reader``

In [7]:
#export


class ParseYaml:
    """
    Simple Parser from project yamls
    Reading the yaml file

    :param path: path to the yaml file
    """

    def __init__(self, path: str, f_logger=logging.getLogger(__name__)):
        self._logger = f_logger
        if os.path.exists(path):
            self.yaml_file = yml_reader(path, self._logger)
        else:
            self.yaml_file = None
            self._logger.info(f'yaml file {path} does not exisit')

    def get_yaml(self, data_list: list = None):
        """
        Return Yaml Query Info

        :param data_list: Optional if None returns full yaml
        allows user to index into yaml file as far as they
        want and index allows.

        Return: specific yaml section
        """
        if not data_list:
            return self.yaml_file
        yaml_key = self.yaml_file.get(f'{data_list[0]}')
        if len(data_list) > 1:
            for level in data_list[1:]:
                yaml_key = yaml_key.get(f'{level}')
        return yaml_key

In [8]:
#export


def yml_reader(yml_filepath: str, logger: logging = None):
    """
    Reading the yaml file.

    :param yml_filepath: path to the yamlfile
    :param logger: logging

    Return: Dictionary of yaml file contents
    """
    _logger = logger if logger is not None else logging.getLogger(__name__)
    if os.path.exists(yml_filepath):
        with open(yml_filepath) as stream:
            yml = YAML(typ="safe")
            yml_dict = yml.load(stream)
        return yml_dict
    else:
        _logger.info(f"yml_filepath ({yml_filepath}) doesn't exisit")

# How to Use

In [9]:
show_doc(ParseYaml)

<h2 id="ParseYaml" class="doc_header"><code>class</code> <code>ParseYaml</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>ParseYaml</code>(**`path`**:`str`, **`f_logger`**=*`<Logger __main__ (INFO)>`*)

Simple Parser from project yamls
Reading the yaml file

:param path: path to the yaml file

In [10]:
yaml_file = "./testing/sdsde_test.yaml"

test_parser = ParseYaml(yaml_file)
assert test_parser is not None, f"parser_yaml should be a parser object {test_parser}"
assert test_parser.yaml_file == test_parser.get_yaml(), 'get_yaml default is to grab the yaml_file in __init__'

In [11]:
yaml_dict = test_parser.yaml_file
assert type(yaml_dict) is dict, f"parser_yaml.yaml_file should be a dict, found {type(yaml_dict)}"

In [12]:
yaml_dict

{'local': {'query': {'blob_path': 'query',
   'account_name': 'Connectsnowflakedata',
   'container_name': 'skierability',
   'file_name': 'skierability_query.txt',
   'sql': 'select * from poc_dev.public.ski_score;',
   'push_file': 'skierability_scores.csv'}},
 'test': {'query': {'blob_path': 'query',
   'account_name': 'snowflakedata',
   'container_name': 'skierability',
   'file_name': 'skierability_query.txt',
   'sql': 'SELECT * FROM poc_dev.public.ski_score LIMIT 10',
   'push_file': 'skierability_scores.csv'}}}

In [13]:
yml_dict = test_parser.get_yaml(['test', 'query'])
keys = []
values = []
for k, v in yml_dict.items():
    keys.append(k)
    values.append(v)
assert len(keys) == 6, f"length is 6 length shown f'{len(keys)}'"
assert len(values) == 6, f"length is 6 length shown f'{len(v)}'"

In [14]:
account_name = test_parser.get_yaml(['test', 'query', 'account_name'])
assert account_name == 'snowflakedata', f"Looking for snowflakedata got {account_name}"

In [15]:
assert os.path.exists(yaml_file), f"yaml file does not exist: {yaml_file}"
parser_yaml = ParseYaml(yaml_file)
assert parser_yaml is not None, f"parser_yaml should be a parser object {parser_yaml}"
yaml = parser_yaml.yaml_file
assert type(yaml) is dict, f"parser_yaml.yaml_file should be a dict, found {type(yaml)}"

# Create

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()